In [ ]:
import torch
import os
import shutil
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split

# Step 1: Load your dataset into PyTorch data structures
# Replace 'YourDataset' with the actual dataset you want to use
# Make sure your dataset is in the form of a PyTorch Dataset object
# (e.g., torchvision.datasets, custom Dataset class, etc.)
transform_list = [
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ]
trans = transforms.Compose(transform_list)

data_root = os.path.join(os.path.abspath('..'), "datasets", "BV", "img")
dataset = ImageFolder(root=data_root)

# Step 2: Shuffle the dataset randomly
shuffle_dataset = True
random_seed = 42
if shuffle_dataset:
    torch.manual_seed(random_seed)
    indices = torch.randperm(len(dataset))

# Step 3: Split the shuffled dataset into train and test subsets
# Replace 'train_ratio' with the desired ratio for the training set (e.g., 0.8 for 80% training data)
train_ratio = 0.9
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Step 4: Create DataLoader objects for train and test subsets
# Set 'batch_size' to the desired batch size you want for training and testing
train_dir = os.path.join(os.path.abspath('..'), "datasets", "BV", "train")
test_dir = os.path.join(os.path.abspath('..'), "datasets", "BV", "test")


for img, label in train_dataset:
    class_name = train_dataset.classes[label]
    class_dir = os.path.join(train_dir, class_name)

    # Create the class directory if it doesn't exist
    os.makedirs(class_dir, exist_ok=True)

    # Save the image with a unique filename in the class directory
    img_filename = f"{label}_{os.path.basename(train_dataset.samples[label][0])}"
    output_path = os.path.join(class_dir, img_filename)
    shutil.copy(train_dataset.samples[label][0], output_path)

print("Data saved to different directories successfully.")
for img, label in test_dataset:
    class_name = test_dataset.classes[label]
    class_dir = os.path.join(test_dir, class_name)

    # Create the class directory if it doesn't exist
    os.makedirs(class_dir, exist_ok=True)

    # Save the image with a unique filename in the class directory
    img_filename = f"{label}_{os.path.basename(test_dataset.samples[label][0])}"
    output_path = os.path.join(class_dir, img_filename)
    shutil.copy(test_dataset.samples[label][0], output_path)
